<a href="https://colab.research.google.com/github/Pradeepsh0000/sleep/blob/main/Copy_of_Copy_of_%22Analyzing_and_Predicting_Sleep_Disorder%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
uom190346a_sleep_health_and_lifestyle_dataset_path = kagglehub.dataset_download('uom190346a/sleep-health-and-lifestyle-dataset')

print('Data source import complete.')

Using Colab cache for faster access to the 'sleep-health-and-lifestyle-dataset' dataset.
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sleep-health-and-lifestyle-dataset/Sleep_health_and_lifestyle_dataset.csv


# "Analyzing and Predicting Sleep Disorder"

## What will you learn from this project?
* EDA
* Categoric and Numeric Features
* Feature Engineering
* Data Standardization and Normalization
* Data Visualization
* ML Modelling , Tuning and Evaluation


## İntroduction
Sleep is a fundamental biological process that significantly impacts the quality of human life. Adequate and quality sleep plays a crucial role in maintaining physical health, mental well-being, and overall life satisfaction. However, many people worldwide suffer from sleep disorders. Sleep disorders manifest in various ways, such as difficulty falling asleep, waking up feeling unrested, excessive daytime sleepiness, or irregular sleep patterns. These disorders not only affect an individual’s physical health but also have negative consequences on mental health and daily functioning.

In recent years, understanding sleep disorders better and developing effective treatment methods have become major research areas in the healthcare sector. In this context, analyzing and predicting sleep disorders can be beneficial both at the clinical and individual levels. With the advancement of technology, analyzing sleep data holds great potential for early detection of sleep disorders and finding effective treatments.

This study aims to analyze sleep disorders and predict the risk of future sleep issues. By utilizing data-driven approaches and machine learning techniques, various factors related to individuals' sleep patterns will be considered to predict the risk of sleep disorders. Through this, it is expected to contribute to early interventions and the development of personalized treatment strategies.


![](https://www.verywellhealth.com/thmb/NYQSoa2QZuOiVged9b5NEGyVrdY=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/VWH-LauraPorter-InsomniavsSleepApnea-Standard-8fa2b9135a3c45dd95ce3177af664176.jpg)

## Data Content
1. Person ID: Unique identifier for each person.
2.  Gender: Gender of the person (Male).
3. Age: Age of the person.
4. Occupation: The job or occupation of the person.
5. Sleep Duration: Number of hours of sleep per day.
6. Quality of Sleep: A rating of the quality of sleep (on a scale of 1 to 10).
7. Physical Activity Level: Amount of physical activity (on a scale of 1 to 10).
8. Stress Level: Level of stress (on a scale of 1 to 10).
9. BMI Category: Classification of Body Mass Index (e.g., Overweight, Normal, Obese).
10. Blood Pressure: Blood pressure value (e.g., 126/83).
11. Heart Rate: Heart rate per minute.
12. Daily Steps: Number of steps walked daily.
13. Sleep Disorder: Information about any sleep disorder, if applicable (e.g., Sleep Apnea).

### Read and Analyze Data


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/My Drive/Diagnosis ML Model/Sleep_health_and_lifestyle_dataset.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Diagnosis ML Model/Sleep_health_and_lifestyle_dataset.csv'

In [ ]:
df.head()

In [ ]:
# describe basic statistics of data
df.describe().T

In [ ]:
# information about data frame
df.info()

### Missing Value and Unique Counts Analysis

In [ ]:
df.isnull().sum()

In [ ]:
df["Stress Level"].value_counts()

In [ ]:
def func(df):

    for column in df.columns:
        print(f"Unique counts for column: {column}")
        print(df[column].unique())
        print()

In [ ]:
func(df)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

In [ ]:
msno.matrix(df)

In [ ]:
df["Sleep Disorder"].fillna("Nothing",inplace=True)

In [ ]:
df[["Sleep Disorder"]].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().T

## Analysis and Visualization

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(hue="Occupation",x="Sleep Disorder",data=df,palette="Set1")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Stress distribution based on occupation")
sns.barplot(y="Occupation",x="Stress Level",data=df,palette="Set2")
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_barplots(df):
    for column in df.columns:
        if column != "Sleep Duration":
            plt.figure(figsize=(10, 10))
            sns.barplot(y=column, x="Sleep Duration", data=df, palette="Set3")
            plt.title(f"Sleep Duration vs {column}")
            plt.show()


In [ ]:
plot_barplots(df);

In [ ]:
sns.lineplot(y="Sleep Disorder",x="Age",data=df);

## Outlier Analysis

In [ ]:
def detect_outliers(df, method="IQR", threshold=1.5):
    outlier_dict = {}

    for col in df.select_dtypes(include=[np.number]):
        if method == "IQR":
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

        elif method == "z-score":
            mean = df[col].mean()
            std = df[col].std()
            z_scores = (df[col] - mean) / std
            outliers = df[np.abs(z_scores) > threshold]

        else:
            raise ValueError("Invalid method! You should use 'IQR' or 'z-score'.")

        outlier_dict[col] = outliers[col].values

        # Visualize outliers
        plt.figure(figsize=(6, 4))
        sns.boxplot(x=df[col])
        plt.title(f"Outliers ({col})")
        plt.show()

    return outlier_dict

outliers = detect_outliers(df, method="IQR", threshold=1.5)

for key, values in outliers.items():
    print(f"Outliers in the {key} column: {values}")


In [ ]:
df = df[~df["Heart Rate"].isin([85, 82, 80, 81, 84, 83, 86])]


In [ ]:
df["Heart Rate"].value_counts()

In [ ]:

df[["Systolic Pressure", "Diastolic Pressure"]] = df["Blood Pressure"].str.split("/", expand=True).astype(int)
df.drop(columns=["Blood Pressure"], inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()


for col in df.select_dtypes(include=["object"]).columns:
    df_encoded[col] = LabelEncoder().fit_transform(df[col])


plt.figure(figsize=(10, 6))
sns.heatmap(df_encoded.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
df = df.drop(columns=["Physical Activity Level","Person ID"])

## ML Modelling , Tuning and Evaluation


In [ ]:
X = df.drop(columns=['Sleep Disorder'])
y = df['Sleep Disorder']

In [ ]:
X = pd.get_dummies(X,drop_first=True)

In [ ]:
X

In [ ]:
le = LabelEncoder()

In [ ]:
y = le.fit_transform(y)

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier


In [ ]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Support Vector Machine": SVC(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(eval_metric='mlogloss'),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Network (MLP)": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
}

In [ ]:
results = {}

from sklearn.metrics import accuracy_score, classification_report

# Train and test all models
for name, model in models.items():
    model.fit(X_train, y_train)  # Train the model
    y_pred = model.predict(X_test)  # Make predictions on the test data

    accuracy = accuracy_score(y_test, y_pred)  # Calculate accuracy score
    results[name] = accuracy  # Store the results

    print(f"{name} - Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))
    print("="*50)


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 1', 'Class 2', 'Class 3'], yticklabels=['Class 1', 'Class 2', 'Class 3'])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix (Multiclass)")
plt.show()
